In [9]:
import os
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings # Ollama向量嵌入器
from langchain_community.chat_models import ChatOllama # ChatOllma聊天模型
from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
# from model_factory import yi_llm
# import PyPDF2
os.environ["OPENAI_API_KEY"] = "ab78b1a21a614ba1ade36ba88b172893"

In [4]:
client = OpenAI(
    api_key='ab78b1a21a614ba1ade36ba88b172893',
    base_url='https://api.lingyiwanwu.com/v1'
)
completion = client.chat.completions.create(
    model="yi-large",
    messages=[{"role": "user", "content": "Hi, who are you?"}]
)

print(completion.choices[0].message.content)

Hello! I am an AI assistant developed by 01.AI, designed to be a helpful resource for you, capable of answering questions and offering insightful information across a wide range of topics. How may I be of service to you today?


In [13]:
bge_model_path = "BAAI/bge-small-zh-v1.5"
root_dir = "./papers"

def extract_file_dirs(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
            # if file.endswith(".txt"):
                fp = os.path.join(root, file)
                file_paths.append(fp)
    return file_paths

files = extract_file_dirs(root_dir)
print(files[0])
print(files[1])
loaders = [TextLoader(file) for file in files]
loaders

./papers\test1.md
./papers\test2.md


In [12]:
docs = []
for l in loaders:
    docs.extend(l.load())
print("************1")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
print("************2")
# huggingface_bge_embeddings = HuggingFaceBgeEmbeddings(model_name=bge_model_path)
# huggingface_bge_embeddings = OllamaEmbeddings(model_name=bge_model_path)
huggingface_bge_embeddings = OpenAIEmbeddings()
print("************3")
vertorstore = Chroma.from_documents(documents, huggingface_bge_embeddings, persist_directory="./vectorstore")
print("************4")

query = "是谁用长刀一刀将仇人的喉咙斩开？"
result = vertorstore.similarity_search(query, k=3)
print("************5")

for doc in result:
    print(doc.page_content)
    print('******')

************1
************2
************3


APITimeoutError: Request timed out.

In [15]:
retriver = vertorstore.as_retriever()
docs = retriver.get_relevant_documents(query)

c:\Users\HHH\.conda\envs\chemical_rag\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [19]:
template = """
    Answer the question based only on the context: 
    {context}
    Question: {question},请用中文输出答案。
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOllama(model="llama3", temperature=10)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.invoke(query)
print(response)
response2 = rag_chain.invoke(query)
print(response)

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015F819B79D0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。'))